In [1]:
#importing dependancies
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
#Build and Train LSTM Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# Getting keypoints from mediapipe holistic
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
     # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
# Writing function with optimised code to get landmark values
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [7]:
# Setting up folders for data collection
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('DATA') 

# Actions that we try to detect
actions = np.array(['A', 'B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U',
                    'V','W','X','Y','Z','BLANK'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
#BUILD MODEL
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [11]:
model.load_weights('SignLanguageDetection330.h5')

In [12]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), -1)
#         print(actions[num]," ",int(prob*100))
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
getAlphaForLong = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
#             print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                    getAlphaForLong.append(actions[np.argmax(res)])
                    print(actions[np.argmax(res)])
#                 if len(sentence) > 0: 
#                     if actions[np.argmax(res)] != sentence[-1]:
#                         sentence.append(actions[np.argmax(res)])
#                 else:
#                     sentence.append(actions[np.argmax(res)])
#                 print(getAlphaForLong)
            if(len(getAlphaForLong)>50):
#                 print(getAlphaForLong)
#                 arr = np.array(getAlphaForLong)
#                 print(max(getAlphaForLong,key=getAlphaForLong.count))
#                 print(arr[np.argmax(arr)])
#                 print(arr)
                if(max(getAlphaForLong,key=getAlphaForLong.count)=='BLANK'): sentence = sentence[:-1]
                else: sentence.append(max(getAlphaForLong,key=getAlphaForLong.count))
                print(getAlphaForLong)
                getAlphaForLong = []

            if len(sentence) > 10: 
                sentence = sentence[-10:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step
O
1/1 [==============================] - 0s 19ms/step
O
1/1 [==============================] - 0s 23ms/step
O
1/1 [==============================] - 0s 22ms/step
O
1/1 [==============================] - 0s 20ms/step
S
1/1 [==============================] - 0s 23ms/step
S
1/1 [==============================] - 0s 20ms/step
S
1/1 [==============================] - 0s 21ms/step
S
1/1 [==============================] - 0s 21ms/step
S
1/1 [==============================] - 0s 19ms/step
S
1/1 [==============================] - 0s 19ms/step
S
1/1 [==============================] - 0s 23ms/step
S
1/1 [==============================] - 0s 19ms/step
S
1/1 [==============================] - 0s 19ms/step
S
1/1 [==============================] - 0s 19ms/step
S
1/1 [==============================] - 0s 21ms/step
S
1/1 [==============================] - 0s 21ms/step
S
1/1 [==============================] - 0s 19ms/step
S
1/1 [=======================

BLANK
1/1 [==============================] - 0s 21ms/step
BLANK
1/1 [==============================] - 0s 21ms/step
BLANK
1/1 [==============================] - 0s 21ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 19ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 19ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 21ms/step
BLANK
1/1 [==============================] - 0s 18ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 19ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [==============================] - 0s 20ms/step
BLANK
1/1 [===